Spotipy API

In [8]:
#Importing neccesary libraries and Cliend Cridentials Flow

import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
#Creating Spotify App and getting ID´s

Client_ID = "c71bae50194549399a615967e53c6450"
Redirect_URI = "http://localhost:8080/callback/"


In [10]:
#Opening the secret file in read mode and assigning the json file to the variable "secret"
with open('Spotify_secret.json', "r") as f:
    secret = json.load(f)

In [11]:
#Reading the secret file with the dictionary object "secret"
secret["secret"]

'20adff09ca8a475ab94c13cede1ee868'

In [12]:
#soptify app keys
sp_auth = spotipy.SpotifyOAuth(client_id = Client_ID,
                      client_secret = secret["secret"],
                      scope = 'user-read-recently-played,user-top-read',
                      redirect_uri = Redirect_URI)

In [13]:
#getting authorizatgion 
sp_auth.get_auth_response()

'AQBs6talxnH4mdwlW6AYL8uHw9ucyzU_ucryHdz718F1FUUxez1YL4BwAS4CzIREdd6P9n9X01NMMp7wQbWx6X9hi1ClSqU2w-JKy7qb2K5t465g8uwRgMY0bsBZ56cBJb5q9WzNV_kxFqkwE8ObHemeFcE6MvvxHnLKh1fAB5HLQulrRcjgPuKhqAj9gAjjpdteE2WIKXT-Ov1ZtpMP_XGZ4p7ZOnLnND_nhRY'

In [14]:
#extra access via "current_user_recently_played()"
sp = spotipy.Spotify(auth_manager = sp_auth)

In [15]:
sp.current_user()

{'display_name': 'Kristin Bjørkhaug Johansen',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1199800973'},
 'followers': {'href': None, 'total': 2},
 'href': 'https://api.spotify.com/v1/users/1199800973',
 'id': '1199800973',
 'images': [{'height': None,
   'url': 'https://scontent-ams2-1.xx.fbcdn.net/v/t1.6435-1/106509755_3120303878035800_7198362888784915788_n.jpg?stp=dst-jpg_s320x320&_nc_cat=105&ccb=1-7&_nc_sid=0c64ff&_nc_ohc=6J4ZOfNNenYAX8tzZFb&_nc_ht=scontent-ams2-1.xx&edm=AP4hL3IEAAAA&oh=00_AfCKaLxo5TxCi6SwhQxwZsl9IBVzX7VADkUytB6wHf3NYQ&oe=64532D7C',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:1199800973'}

In [16]:
# work with data
my_top_songs = sp.current_user_top_tracks(limit = 20, time_range = "long_term")


In [17]:
dl_top = []
    
for item in my_top_songs["items"]:
    d = {}
    d["track_title"] = item["name"]
    da = []
    for artist in item["artists"]:
        da.append(artist["name"])
    d["artists"] = da
    d["popularity"] = item["popularity"]
    d["uri"] = item["uri"]
    d["duration"] = item["duration_ms"]
    dl_top.append(d)
my_top_songs_df = pd.DataFrame(dl_top)

In [18]:
#Save dataset for top songs
my_top_songs_df.to_csv("Data/Top_20_tracks.csv")
Dataset_1 = pd.read_csv("Data/Top_20_tracks.csv")

In [19]:
features = sp.audio_features(my_top_songs_df['uri'])

In [20]:
Df_features = pd.DataFrame(features)

In [21]:
#Save dataset for features
Df_features.to_csv("Data/Top_song_features.csv")

In [22]:
Dataset_2 = pd.read_csv("Data/Top_song_features.csv")

In [27]:
my_top_artists = sp.current_user_top_artists(limit=20, offset=0, time_range='long_term')

In [38]:
# extract the genre information for each artist and add it to a list
genres_list = []
for artist in my_top_artists['items']:
    genres_list.extend(artist['genres'])

# count the frequency of each genre
genre_counts = {}
for genre in genres_list:
    if genre not in genre_counts:
        genre_counts[genre] = 1
    else:
        genre_counts[genre] += 1

# sort the genre counts and take the top 10
top_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)[:10]

# create a DataFrame from the top genres data
df = pd.DataFrame(top_genres, columns=['genre', 'count'])

# save the DataFrame to a CSV file
df.to_csv("Data/Top_genres.csv", index=False)